In [2]:
import numpy as np
import os
import random
import PIL
from PIL import Image
from tqdm import tqdm
from patchify import patchify

root = r'C:\Users\james\Documents\parcel_boundary_data\raw_data'

folders = ['chicago','houston','newyork_boston','pheonix_arizona','seattle']

In [3]:
# method to crop collected image data
def crop_image(root,location,type): 
    # root path of data
    location_path = os.path.join(root,location)
    # select image or mask
    path = os.path.join(location_path,type)
    # sorted list of image paths
    image_path = sorted([ os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(".png") ])
    # loop through image paths, tqdm provides a progress to understand computation progess
    for img_path in tqdm(image_path):
        # open image at specific image path
        img = Image.open(img_path)
        # covert image to RGB
        img = img.convert('RGB')
        # specific crop coordinates
        img = img.crop((44, 192, 2465, 1284))
        # save image
        img.save(img_path)
    return
    
# function to loop through image to create binary parcel boundary mask
def extract_parcel_boundaries(root,location):
    location_path = os.path.join(root,location)
    path = os.path.join(location_path,'mask')
    image_path = sorted([ os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(".png") ])
    for img_path in tqdm(image_path):
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = create_boundaries(img)
        img.save(img_path)
    return

# function to create binary mask from original image
def create_boundaries(img):
    # loop from each pixel in image width then height
    for width_trav in range(img.size[0]):
        for height_trav in range(img.size[1]):
            pixel = img.getpixel((width_trav,height_trav))
            # if R pixel value > 130 and B pixel value > 130 reassign pixel as white
            if pixel[1] >= 130 and pixel[2] >= 130:
                img.putpixel((width_trav,height_trav),(255,255,255))
            # else reassign black
            else:
                img.putpixel((width_trav,height_trav),(0,0,0))
    return img

# create image patches for single image
def create_patches(img_path,patch_size,target_save_location):
    head, tail = os.path.split(os.path.splitext(img_path)[0])
    img = Image.open(img_path)
    img = (np.array(img.convert('RGB')))
    size_x = (img.shape[0] // patch_size) * patch_size  
    size_y = (img.shape[1] // patch_size) * patch_size 
    img_patches = np.squeeze(patchify(np.array(img[0:size_x, 0:size_y]), (patch_size, patch_size, 3), step=patch_size))
    count = 1 
    for x in range(img_patches.shape[0]):
        for y in range(img_patches.shape[1]):
            patch = img_patches[x,y]
            img = Image.fromarray(patch)
            img.save((target_save_location + '\\' + tail + '_patch_' + str(count) + '.png'))
            count += 1

# loop through whole dataset and patch each image
def patch_images(root,type,target,patch_size):
    path = os.path.join(root,type)
    image_path = sorted([ os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(".png") ])
    for i in tqdm(image_path):
        create_patches(i,patch_size,target)
    return image_path

In [ ]:
extract_parcel_boundaries(r'C:\Users\james\Documents\parcel_boundary_data\raw_data\data_by_location','la')

In [15]:
create_patches(r'C:\Users\james\Documents\parcel_boundary_data\raw_data\data_by_type\image\chicago_boundary_1_image.png',256,r'C:\Users\james\Documents\parcel_boundary_data\patched_data\256\image')

In [ ]:
patch_images(r'C:\Users\james\Documents\parcel_boundary_data\raw_data\data_by_type','image',r'C:\Users\james\Documents\parcel_boundary_data\patched_data\256\image',256)